In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.applications import VGG19
from tensorflow.keras.utils import to_categorical

# Function to generate synthetic dataset
def generate_dataset(samples=1000):
    spo2 = np.random.uniform(85, 100, samples)
    heart_rate = np.random.uniform(60, 120, samples)
    labels = np.select(
        [spo2 < 90, (spo2 >= 90) & (spo2 < 95), spo2 >= 95],
        [2, 1, 0]  # 2: Severe Hypoxia, 1: Mild Hypoxia, 0: Normal
    )
    dataset = pd.DataFrame({'SpO2': spo2, 'HeartRate': heart_rate, 'Label': labels})
    return dataset

# Generate and save the dataset
dataset = generate_dataset(3000)
dataset.to_csv('pulseoximeter_dataset.csv', index=False)

# Load the dataset
data = pd.read_csv('pulseoximeter_dataset.csv')

# Split into features (X) and labels (y)
X = data[['SpO2', 'HeartRate']].values
y = data['Label'].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Save the scaler for later use
joblib.dump(scaler, 'scaler.pkl')

# Function to convert tabular data to "images"
def tabular_to_image(data, img_size=(64, 64)):
    return np.array([np.full(img_size, row.mean()) for row in data]).reshape(-1, 64, 64, 1)

X_train_images = tabular_to_image(X_train_scaled)
X_test_images = tabular_to_image(X_test_scaled)

# Define the VGG19 model for tabular data
def create_vgg19_model(input_shape=(64, 64, 1), num_classes=3):
    base_model = VGG19(weights=None, include_top=False, input_shape=input_shape)
    x = Flatten()(base_model.output)
    x = Dense(128, activation='relu')(x)
    x = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=x)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Create the model
vgg19_model = create_vgg19_model(input_shape=(64, 64, 1), num_classes=3)

# Train the model
history = vgg19_model.fit(
    X_train_images,
    to_categorical(y_train),
    epochs=10,
    batch_size=32,
    validation_data=(X_test_images, to_categorical(y_test))
)

# Save the trained model
vgg19_model.save('models/pulse_vgg19_model.h5')


Epoch 1/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 150s 2s/step - accuracy: 0.3305 - loss: 1.7296 - val_accuracy: 0.3517 - val_loss: 1.0984
Epoch 2/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 146s 2s/step - accuracy: 0.3578 - loss: 1.0983 - val_accuracy: 0.3517 - val_loss: 1.0981
Epoch 3/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 182s 2s/step - accuracy: 0.3499 - loss: 1.0984 - val_accuracy: 0.3517 - val_loss: 1.0979
Epoch 4/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 251s 3s/step - accuracy: 0.3502 - loss: 1.1083 - val_accuracy: 0.3517 - val_loss: 1.0664
Epoch 5/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 252s 3s/step - accuracy: 0.4024 - loss: 1.1377 - val_accuracy: 0.3517 - val_loss: 1.0983
Epoch 6/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 245s 3s/step - accuracy: 0.3436 - loss: 1.0991 - val_accuracy: 0.3517 - val_loss: 1.0979
Epoch 7/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 248s 3s/step - accuracy: 0.3422 - loss: 1.0987 - val_accuracy: 0.3517 - val_loss: 1.0979
Epoch 8/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 260s 3s/step - accuracy: 0.3468 - loss: 1.0983 - val_accuracy: 0.3517 - v